---
---

# Arxiv Consultant

An intelligent agent system that can interactively answer queries about academic papers fetched from arXiv. This system is particularly designed to assist with Intellectual Property (IP) consulting by leveraging language models and various tools to provide accurate and relevant information strictly based on the fetched documents.

---
---

## Table of Content

* [Setup the project](#setup)
    * [Install required libraries](#install)
    * [Import required libraries](#import)
    * [Define Globals](#define)
* [Build tools using papers fetched from Arxiv database](#builder)
    * [PaperAgentBuilder](#paperbuilder)
    * [ArxivBuilder](#arxivbuilder)
* [Retrieve relavant object](#retriever)
* [Build Retriever-Enabled OpenAI Agent](#top_agent)
* [Arxiv Consultant](#consultant)
* [Playground](#playground)

---
---

<a id='setup'></a>
## <p style="color: #ffd700; background-color: #006400;"> Setup the project
<a id='install'></a>
### Install required libraries

In [1]:
%pip install llama-index==0.10.50

Note: you may need to restart the kernel to use updated packages.


---

<a id='import'></a>
### Import required libraries

In [2]:
# Import required libraries
# Necessary libraries
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
import os
from typing import Optional

# Readers and settings
from llama_index.core import Settings
from llama_index.core.readers import SimpleDirectoryReader
# Core components
from llama_index.core import load_index_from_storage, StorageContext, VectorStoreIndex, SummaryIndex
from llama_index.core.objects import ObjectIndex, ObjectRetriever
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.schema import QueryBundle
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.agent import ReActAgent
# Post processor
from llama_index.core.postprocessor import SentenceTransformerRerank
# Open AI components
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.agent.openai import OpenAIAgent

---

<a id='define'></a>
### Define Globals

In [3]:
# Setup Open AI Api Key
OPEN_AI_API_KEY = "<Your Key>"
# Create llm model with Open AI API Key
llm = OpenAI(model="gpt-3.5-turbo", api_key=OPEN_AI_API_KEY)
Settings.llm = llm
# Create embedding model
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", embed_batch_size=256, api_key=OPEN_AI_API_KEY
)

# Set the path for storing the required data
storage_path='./storage'

In [4]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio
nest_asyncio.apply()

# Suppress warning logs
import warnings
warnings.filterwarnings("ignore")

---
---
<a id='builder'></a>
## <p style="color: #ffd700; background-color: #006400;"> Build tools using papers fetched from Arxiv database

<a id='paperbuilder'></a>
- **PaperAgentBuilder** class is structured to handle document parsing, indexing, querying, and agent creation efficiently, ensuring that both vector-based and summarization-based querying capabilities are available for each document.

In [5]:
class PaperAgentBuilder():
    """
     A class to build agent with vector query engine and summary index tool for a single paper in pdf
    """
    def __init__(self):
        # Initialize the class with a node parser, which is an instance of SentenceSplitter.
        self.node_parser = SentenceSplitter()

    def build_agent_per_doc(self, nodes, paper_name):
        """
        Build an agent for a single document.

        Parameters:
        nodes (list): List of nodes (parsed sentences or segments) from the document.
        paper_name (str): Name of the paper.

        Returns:
        tuple: (agent, summary) where agent is the built agent and summary is the document summary.
        """
        # Define paths for storing vector index and summary
        vector_index_out_path = f"{storage_path}/vector/{paper_name}"
        summary_out_path = f"{storage_path}/summary/{paper_name}_summary.pkl"
        
        # Check if vector index already exists, if not, create and store it
        if not os.path.exists(vector_index_out_path):
            Path(storage_path).mkdir(parents=True, exist_ok=True)
            # Build vector index using the provided nodes
            vector_index = VectorStoreIndex(nodes)
            vector_index.storage_context.persist(persist_dir=vector_index_out_path)
        else:
            # Load the existing vector index from storage
            vector_index = load_index_from_storage(
                StorageContext.from_defaults(persist_dir=vector_index_out_path),
            )

        # Build summary index from the nodes
        summary_index = SummaryIndex(nodes)

        # Define query engines using the indices and the language model (llm)
        vector_query_engine = vector_index.as_query_engine(llm=llm)
        summary_query_engine = summary_index.as_query_engine(
            response_mode="tree_summarize", llm=llm
        )

        # Extract a summary of the document and store it if not already done
        if not os.path.exists(summary_out_path):
            Path(summary_out_path).parent.mkdir(parents=True, exist_ok=True)
            summary = str(
                summary_query_engine.query(
                    "Extract a concise 5-6 lines summary of this document"
                )
            )
            pickle.dump(summary, open(summary_out_path, "wb"))
        else:
            summary = pickle.load(open(summary_out_path, "rb"))

        # Define tools for querying
        query_engine_tools = [
            QueryEngineTool(
                query_engine=vector_query_engine,
                metadata=ToolMetadata(
                    name=f"vector_tool_{paper_name}",
                    description=f"Useful for questions related to specific facts",
                ),
            ),
            QueryEngineTool(
                query_engine=summary_query_engine,
                metadata=ToolMetadata(
                    name=f"summary_tool_{paper_name}",
                    description=f"Useful for summarization questions",
                ),
            ),
        ]

        # Build the agent with the defined tools and a specific system prompt
        agent = OpenAIAgent.from_tools(
            tools=query_engine_tools,
            llm=llm,
            verbose=True,
            system_prompt=f"""\
    You are a specialized agent designed to answer queries about the `{paper_name}`.
    Be intelligent to choose the appropriate tool for answering the query. 
    Use vector_tool_{paper_name} for answering generic query.
    Use summary_tool_{paper_name} for answering questions related to summarization.
    You must ALWAYS use at least one of the tools provided when answering a question;
    Do NOT rely on prior knowledge else you will be heavily penalized.
    """,
        )

        return agent, summary

    def build_agents(self, folder_path, paper_lookup):
        """
        Build agents for multiple documents.

        Parameters:
        folder_path (str): Path to the folder containing the documents.
        paper_lookup (dict): Dictionary mapping filenames to metadata.

        Returns:
        tuple: (agents_dict, extra_info_dict) where agents_dict contains agents for each paper
               and extra_info_dict contains summaries and nodes for each paper.
        """
        def get_paper_metadata(filename):
            # Helper function to get metadata for a paper given its filename
            return paper_lookup[os.path.basename(filename)]

        # Dictionaries to hold the agents and additional information
        agents_dict = {}
        extra_info_dict = {}
        
        # Iterate through all papers and build agents
        for idx, paper_name in enumerate(tqdm(paper_lookup, desc='Building Agents', total=len(paper_lookup))):
            # Create file path for each paper
            paper_file_path = os.path.join(folder_path, paper_name)
            
            # Load the document using a directory reader and get the nodes
            reader = SimpleDirectoryReader(
                input_files=[paper_file_path],
                file_metadata=get_paper_metadata,
            ).load_data()
            nodes = self.node_parser.get_nodes_from_documents(reader)

            # Build an agent for the document
            agent, summary = self.build_agent_per_doc(nodes, paper_name[:-4])

            # Store the agent and additional info in the dictionaries
            agents_dict[paper_name] = agent
            extra_info_dict[paper_name] = {"summary": summary, "nodes": nodes}

        return agents_dict, extra_info_dict


---

<a id='arxivbuilder'></a>
- **ArxivBuilder** class is structured to automate the process of fetching research papers from Arxiv, storing metadata, downloading the papers, creating querying agents for each paper using *PaperAgentBuilder*, and managing the persistence of downloaded files.    

In [6]:
class ArxivBuilder():
    """
    A class to handle fetching papers from Arxiv, downloading them, and building agents for each paper.
    """

    def __init__(
        self,
        search_query: str,
        persist: Optional[bool] = False,
        papers_dir: Optional[str] = ".papers",
        max_results: Optional[int] = 10,
    ) -> None:
        """
        Initialize with parameters.
        
        Args:
            search_query (str): A topic to search for (e.g. "Artificial Intelligence").
            persist (Optional[bool]): Whether to store the paper on local disk.
            papers_dir (Optional[str]): Local directory to store the papers.
            max_results (Optional[int]): Maximum number of papers to fetch.
        """
        self.agent_builder = PaperAgentBuilder()
        self.agents_dict = {}
        self.extra_info_dict = {}
        self.paper_lookup = {}
        self.search_query = search_query
        self.persist = persist
        self.papers_dir = papers_dir
        self.max_results = max_results
        self.tools = []

    def download_paper_and_build_agent(self):
        """
        - Search for a topic on Arxiv.
        - Download the PDFs of the top results locally then read them.
        - Build an agent for each paper.
        """
        import arxiv
        
        # Search for papers on Arxiv using the specified query and parameters.
        arxiv_search = arxiv.Search(
            query=self.search_query,
            id_list=[],
            max_results=self.max_results,
            sort_by=arxiv.SortCriterion.Relevance,
        )
        search_results = list(arxiv_search.results())
        print(f"> Successfully fetched {len(search_results)} papers")
            

        # Ensure the directory to store papers exists.
        if not os.path.exists(self.papers_dir):
            os.makedirs(self.papers_dir)

        # Prepare a dictionary to hold paper metadata and download the papers.
        self.paper_lookup = {}
        for paper in search_results:
            # Create a filename from the paper's entry ID
            filename = paper.entry_id.split('/')[-1].replace('.', '_') + '.pdf'
            # Store paper metadata in the lookup dictionary.
            self.paper_lookup[filename] = {
                "Title of this paper": paper.title,
                "Authors": ", ".join([a.name for a in paper.authors]),
                "Date published": paper.published.strftime("%m/%d/%Y"),
                "URL": paper.entry_id,
                "summary": paper.summary
            }
            # Download the paper PDF.
            paper.download_pdf(dirpath=self.papers_dir, filename=filename)
            print(f"> Downloading {filename}...")

        # Build agents for the downloaded papers.
        self.agents_dict, self.extra_info_dict = self.agent_builder.build_agents(self.papers_dir, self.paper_lookup)

        # If persistence is not required, delete the downloaded papers and directory.
        if not self.persist:
            try:
                for f in list(self.paper_lookup.keys()):
                    os.remove(os.path.join(self.papers_dir, f))
                    print(f"> Deleted file: {f}")
                os.rmdir(self.papers_dir)
                print(f"> Deleted directory: {self.papers_dir}")
            except OSError:
                print("Unable to delete files or directory")

    def get_tools(self):
        """
        Get the tools (agents) created for each paper.
        
        Returns:
            list: List of QueryEngineTool instances for each paper.
        """
        # Reset the tools list
        self.tools = []
        # Iterate through each agent and build tool
        for file_name, agent in self.agents_dict.items():
            summary = self.extra_info_dict[file_name]["summary"]
            # Create a QueryEngineTool for each agent, with the corresponding summary as metadata.
            doc_tool = QueryEngineTool(
                query_engine=agent,
                metadata=ToolMetadata(
                    name=f"tool_{file_name[:-4]}",  # Use the file name without extension.
                    description=summary,  # Use the paper's summary as the description.
                ),
            )
            # Add the tool to the list of tools.
            self.tools.append(doc_tool)

        return self.tools

    def run(self):
        """
        Execute the process of downloading papers, building agents, and returning the tools.

        Returns:
            list: A list of QueryEngineTool instances for querying the papers.
        """
        try:
            # Attempt to download papers and build agents
            self.download_paper_and_build_agent()

            # Retrieve and return tools for querying
            return self.get_tools()

        except Exception as e:
            # Handle any exceptions that occur during the process
            print(f"> Not able to download any paper: {e}")
            return None

---

In [7]:
# Provide a Wrong search term to see if it is handling
example_builder = ArxivBuilder(search_query='Latest trends in Physics Science', max_results=2)
example_tools = example_builder.run()

> Successfully fetched 2 papers
> Not able to download any paper: HTTP Error 404: Not Found


In [8]:
# Provide a Valid search term
example_builder = ArxivBuilder(search_query='Latest trends in Computer Science', max_results=2)
example_tools = example_builder.run()

> Successfully fetched 2 papers
> Downloading 1404_0708v1.pdf...
> Downloading 2401_03865v3.pdf...


Building Agents:   0%|          | 0/2 [00:00<?, ?it/s]

> Deleted file: 1404_0708v1.pdf
> Deleted file: 2401_03865v3.pdf
Unable to delete files or directory


In [9]:
# An example tool data
example_tools[0].metadata

ToolMetadata(description='The document discusses the integration of modelling, simulation, and optimization in modern design practices, highlighting the progress made in these areas while addressing unresolved challenges. It emphasizes the use of nature-inspired algorithms and surrogate-based techniques for modelling and optimization. The paper reviews recent trends, challenges, and important topics for further research, including parameter-tuning, large-scale problems, and bridging the gaps between theory and applications. The focus is on computational optimization, modelling, and simulation, showcasing the latest developments and applications in science, engineering, and industry.', name='tool_1404_0708v1', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

---
---
<a id='retriever'></a>
## <p style="color: #ffd700; background-color: #006400;"> Retrieve relavant object
   
- **RelavantObjectRetriever** class is designed to manage the retrieval, processing, and querying of relevant objects (e.g., research papers) efficiently, ensuring that specialized tools are available for different types of queries, especially comparisons.

In [10]:
class RelavantObjectRetriever(ObjectRetriever):
    """
    A class to retrieve relevant objects based on a query, process the nodes, and map them to tools.
    """
    def __init__(
        self,
        retriever,
        object_node_mapping,
        node_postprocessors=None,
        llm=None,
        builder=None
    ):
        """
        Initialize the RelavantObjectRetriever with necessary components.

        Args:
            retriever: An instance used to retrieve nodes based on a query.
            object_node_mapping: A mapping from nodes to objects/tools.
            node_postprocessors (Optional[list]): List of postprocessors to process nodes after retrieval.
            llm (Optional): Language model to use for querying.
        """
        self._retriever = retriever
        self._object_node_mapping = object_node_mapping
        self._llm = llm or OpenAI(model="gpt-3.5-turbo", api_key="")  # Default to GPT-3.5 model if none provided.
        self._node_postprocessors = node_postprocessors or []  # Use provided postprocessors or an empty list.
        self._builder = builder

    def retrieve(self, query_bundle):
        """
        Retrieve relevant objects based on a query, process nodes, and return tools for querying.

        Args:
            query_bundle: Query or QueryBundle object containing the query string.

        Returns:
            list: A list of QueryEngineTool objects including a sub-question tool for comparisons.
        """
        # Convert string query to QueryBundle if necessary.
        if isinstance(query_bundle, str):
            query_bundle = QueryBundle(query_str=query_bundle)

        # Retrieve nodes using the provided retriever.
        nodes = self._retriever.retrieve(query_bundle)

        # Postprocess the retrieved nodes using the provided postprocessors.
        for processor in self._node_postprocessors:
            nodes = processor.postprocess_nodes(nodes, query_bundle=query_bundle)

        # Map nodes to tools using the object node mapping.
        tools = [self._object_node_mapping.from_node(n.node) for n in nodes]

        # Create a sub-question engine for comparison queries.
        sub_question_engine = SubQuestionQueryEngine.from_defaults(
            query_engine_tools=tools, llm=self._llm
        )
        sub_question_description = """\
Useful for any queries that involve comparing multiple documents. ALWAYS use this tool for comparison queries - make sure to call this \
tool with the original query. Do NOT use the other tools for any queries involving multiple documents.
"""
        # Define a tool for sub-question queries with metadata.
        sub_question_tool = QueryEngineTool(
            query_engine=sub_question_engine,
            metadata=ToolMetadata(
                name="compare_tool", description=sub_question_description
            ),
        )

        # Get paper names from the tools and retrieve all relevant nodes from the ArxivBuilder instance.
        paper_names = [tool.metadata.name.replace('tool_', '') + '.pdf' for tool in tools]
        all_nodes = []
        for name in paper_names:
            all_nodes.extend(self._builder.extra_info_dict[name]['nodes'])

        # Build a base index from all nodes and create a query engine for it.
        base_index = VectorStoreIndex(all_nodes)
        base_query_engine = base_index.as_query_engine(llm=self._llm, similarity_top_k=4)
        base_query_engine_tool = QueryEngineTool(
            query_engine=base_query_engine,
            metadata=ToolMetadata(
                name="base_query_engine",
                description="Contains the information of all the related documents in one tool",
            ),
        )

        # Return the list of tools including the sub-question tool and the base query engine tool.
        return tools + [sub_question_tool, base_query_engine_tool]


---

The below setup ensures that relevant objects (e.g., research papers) are retrieved and reranked efficiently, providing the most relevant results based on the specified query. The combination of vector-based retrieval and cross-encoder reranking enhances both the accuracy and relevance of the retrieved results.

In [11]:
# Number of top results to retrieve and rerank
top_n_results = 5

def create_object_retriever(builder):
    """
    Create an ObjectRetriever instance using tools from ArxivBuilder.

    Args:
        arxiv_builder (ArxivBuilder): An instance of ArxivBuilder containing tools and data.

    Returns:
        RelavantObjectRetriever: An ObjectRetriever instance configured with tools and postprocessors.
    """
    # We choose a model with relatively high speed and decent accuracy.
    # This model will rerank the top retrieved results using a cross-encoder model.
    rerank_postprocessor = SentenceTransformerRerank(
        model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=top_n_results
    )

    # Create an ObjectIndex from the tools obtained from the ArxivBuilder instance.
    # The index_cls parameter specifies that we are using a VectorStoreIndex.
    obj_index = ObjectIndex.from_objects(
        builder.get_tools(),
        index_cls=VectorStoreIndex,
    )

    # Convert the ObjectIndex into a node retriever that retrieves nodes based on similarity.
    # The similarity_top_k parameter specifies the number of top similar nodes to retrieve.
    vector_node_retriever = obj_index.as_node_retriever(
        similarity_top_k=top_n_results,
    )

    # Wrap the node retriever with an ObjectRetriever to return objects instead of nodes.
    # The ObjectRetriever takes the following parameters:
    # - retriever: The node retriever created above.
    # - object_node_mapping: The mapping from nodes to objects, obtained from the ObjectIndex.
    # - node_postprocessors: A list of postprocessors to apply to the retrieved nodes, in this case, the rerank_postprocessor.
    # - llm: The language model to use for querying, assumed to be defined elsewhere in the code.
    obj_retriever = RelavantObjectRetriever(
        retriever=vector_node_retriever,
        object_node_mapping=obj_index.object_node_mapping,
        node_postprocessors=[rerank_postprocessor],
        llm=llm,  # llm is assumed to be defined elsewhere
        builder=builder
    )
    
    return obj_retriever


---
---

<a id='top_agent'></a>
## <p style="color: #ffd700; background-color: #006400;"> Build Retriever-Enabled OpenAI Agent

We build a top-level agent that can orchestrate across the different document agents to answer any user query.

In [12]:
def build_arxiv_agent(arxiv_builder):
    """
    Build an instance of ReActAgent using tools provided by RelavantObjectRetriever.

    Args:
        arxiv_builder (ArxivBuilder): An instance of ArxivBuilder containing tools and data.

    Returns:
        ReActAgent: An instance of ReActAgent configured to respond to queries based on the provided tools.
    """
    # Create an instance of ReActAgent using the tools created using arxiv_builder.
    # The tool_retriever parameter initializes the tool retriever with tools for object retrieval.
    # The system_prompt parameter provides instructions on how the agent should respond to queries.
    agent = ReActAgent.from_tools(
        tool_retriever=create_object_retriever(arxiv_builder),
        system_prompt=""" \
    You are an IP consultant designed to answer queries about the documentation.
    ALWAYS use ONLY the AVAILABLE tools provided to answer a question. Do not rely on ANY PRIOR knowledge.
    You are responsible for the following tasks based on user query:
    1) Check if the user idea is relevant across any of the documents.
    2) If you are not able to provide answer with existing knowledge, respond with a note something like
      "The query looks like not in my knowledge. It might either be novel or my knowledge is limited"
    3) Strictly adhere to the provided knowledge, else you will be heavily PENALIZED.\
    """,
        llm=llm,  # The language model to use, assumed to be defined elsewhere in the code.
        verbose=True,  # Enable verbose mode for detailed logging.
        max_iterations=20, # Set the maximum iteration of the agent to 20 for more deeper search.
    )
    
    return agent


---
---

<a id='consultant'></a>
## <p style="color: #ffd700; background-color: #006400;"> Arxiv Consultant
    
This code segment enables the user to search for academic papers on Arxiv, interact with an intelligent agent built to respond to queries based on the retrieved papers, and handles user inputs until the user decides to exit. It provides a structured approach to accessing and utilizing information from academic literature efficiently.

In [13]:
def ArxivConsultant(max_results=10):
    # Prompt the user to provide a search query term for Arxiv.
    text_input = input("Provide the arxiv search term: ")

    # Provide a message indicating the user should wait, as fetching papers might take time.
    print("\nPlease wait, this might take a few minutes...")
    print("-"*50)

    # Initialize an instance of ArxivBuilder to search for papers matching the query term.
    arxiv_builder = ArxivBuilder(search_query=text_input, persist=True, max_results=max_results)

    # Execute the ArxivBuilder to download papers, build agents, and retrieve tools.
    tools = arxiv_builder.run()

    if tools is not None:
        # If tools were successfully retrieved, print a section divider.
        print("-"*50)
        print("Papers I have knowledge on")

        # Iterate over the papers retrieved by ArxivBuilder and print their titles.
        for paper in arxiv_builder.paper_lookup.values():
            print(f"> {paper['Title of this paper']}")

        # Print another section divider for clarity.
        print("-"*50)
        print("> Type 'exit' to close the chat <")
        print("-"*50)
        
        # Build the arxiv_agent using tools obtained from arxiv_builder.
        arxiv_agent = build_arxiv_agent(arxiv_builder)

        # Reset the state of arxiv_agent to ensure it starts fresh for the conversation.
        arxiv_agent.reset()

        # Start an infinite loop to interact with the user until they choose to exit.
        while True:
            print("*"*50)
            # Prompt the user for input and wait for their response.
            text_input = input("User: ")

            # Check if the user wants to exit the interaction.
            if text_input == "exit":
                break  # Exit the loop if the user inputs "exit".

            # Send the user's input to arxiv_agent for processing and get the response.
            response = arxiv_agent.chat(text_input)

            # Print section divider for clarity.
            print("*"*50)

            # Print the response generated by arxiv_agent.
            print(f"Agent: {response}")
    else:
        # If no tools were retrieved (possibly due to search failure), notify the user.
        print("> Retry using a different search term.")
        print("> Exit...")
        print("-"*50)


---
---

<a id='playground'></a>
## <p style="color: #ffd700; background-color: #006400;"> Playground
    
Run the below cell to interact with the top agent

In [16]:
# Modify the 'max_results' value to alter the count of relevant papers to be downloaded
ArxivConsultant(max_results=10)

Provide the arxiv search term: Applications using LLM

Please wait, this might take a few minutes...
--------------------------------------------------
> Successfully fetched 10 papers
> Downloading 2406_10300v1.pdf...
> Downloading 2405_19888v1.pdf...
> Downloading 2404_14809v1.pdf...
> Downloading 2307_09793v1.pdf...
> Downloading 2402_14805v1.pdf...
> Downloading 2310_10049v1.pdf...
> Downloading 2405_14748v1.pdf...
> Downloading 2402_08806v1.pdf...
> Downloading 2406_00936v1.pdf...
> Downloading 2406_09834v1.pdf...


Building Agents:   0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------
Papers I have knowledge on
> Large Language Models as Software Components: A Taxonomy for LLM-Integrated Applications
> Parrot: Efficient Serving of LLM-based Applications with Semantic Variable
> A Survey of Large Language Models on Generative Graph Analytics: Query, Learning, and Applications
> On the Origin of LLMs: An Evolutionary Tree and Graph for 15,821 Large Language Models
> Identifying Multiple Personalities in Large Language Models with External Evaluation
> FATE-LLM: A Industrial Grade Federated Learning Framework for Large Language Models
> MultiCast: Zero-Shot Multivariate Time Series Forecasting Using LLMs
> Combining Insights From Multiple Large Language Models Improves Diagnostic Accuracy
> A Survey of Useful LLM Evaluation
> How and Why LLMs Use Deprecated APIs in Code Completion? An Empirical Study
--------------------------------------------------
> Type 'exit' to close the chat <
-----------------------------------

[tool_2406_09834v1] A: Large Language Models (LLMs) struggle with using up-to-date APIs in code completion for computer vision applications due to their limited ability to understand the context and requirements of specialized domains like computer vision. LLMs may not have been trained on the latest APIs or may lack the specific knowledge required for computer vision tasks, leading to inaccurate or incomplete code suggestions. Additionally, LLMs may struggle with understanding the nuances of computer vision concepts and may provide generic or outdated recommendations that do not align with the current best practices in the field.
[tool_2402_08806v1] A: Aggregating responses from multiple Large Language Models (LLMs) can improve diagnostic accuracy in computer vision applications for guiding blind people by leveraging the diversity of responses. This approach helps mitigate individual model biases and errors, leading to more robust and accurate predictions. By combining outputs from va

User: exit


---
---